# Qwen2VL

In [2]:
!mkdir data
!unzip -q 'ДЛЯ ПИЛОТА.zip' -d  data
!mv data/'ДЛЯ ПИЛОТА' data/pdf

## Convert pdf to image

```bash
pip install -q pyvips==2.2.3
sudo apt install libvips --fix-missing
```

In [17]:
import nltk

import pandas as pd
import numpy as np

import os, glob, re, json

from tqdm.notebook import tqdm
import pyvips

from transformers import AutoProcessor
from vllm import LLM, SamplingParams
from qwen_vl_utils import process_vision_info


RANDOM_SEED = 42
DPI = 150

LABELS_PATH = './data/labels/'
PDF_PATH = './data/pdf/'

PROCESSED_LABELS_PATH = './result/'
IMAGES_PATH = './data/images/'

if not os.path.exists(PROCESSED_LABELS_PATH):
    os.makedirs(PROCESSED_LABELS_PATH)
    
if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH)

In [2]:
pdf_paths = pd.DataFrame(glob.glob(os.path.join(PDF_PATH, '*.pdf')), columns=['pdf_path'])
pdf_paths['pdf_name'] = pdf_paths['pdf_path'].apply(lambda x: x.split('/')[-1])
pdf_paths['pdf_index'] = pdf_paths.index.to_series().apply(lambda x: f'{x:05d}')
pdf_paths

,pdf_path,pdf_name,pdf_index
0,./data/pdf/СП ТОМСК ВЕСЕЛОВА ЕКАТЕРИНА МИХАЙЛО...,СП ТОМСК ВЕСЕЛОВА ЕКАТЕРИНА МИХАЙЛОВНА_ПНН8516...,00000
1,./data/pdf/МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВН...,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,00001
2,./data/pdf/МСК СП ВЕРНИКОВСКИЙ ИВАН ФЁДОРОВИЧ_...,МСК СП ВЕРНИКОВСКИЙ ИВАН ФЁДОРОВИЧ_DNS354368!0...,00002
3,./data/pdf/СП ТОМСК ПНФИЛОВ АЛЕКСЕЙ ВАСИЛЬЕВИЧ...,СП ТОМСК ПНФИЛОВ АЛЕКСЕЙ ВАСИЛЬЕВИЧ_003143972!...,00003
4,./data/pdf/СП ТОМСК ТОИРОВА ИЛЬВИНА РИШАТОВНА_...,СП ТОМСК ТОИРОВА ИЛЬВИНА РИШАТОВНА_009561293!1...,00004
...,...,...,...
994,./data/pdf/СП ТОМСК МАЗИТОВ РОБЕРТ ИРЕКОВИЧ_00...,СП ТОМСК МАЗИТОВ РОБЕРТ ИРЕКОВИЧ_000260665!102...,00994
995,./data/pdf/СП ТОМСК МИТРОХИНА АННА АНДРЕЕВНА_П...,СП ТОМСК МИТРОХИНА АННА АНДРЕЕВНА_ПНН233506!81...,00995
996,./data/pdf/МСК СП БЕЛЯЕВА АНАСТАСИЯ НАГИРОВНА_...,МСК СП БЕЛЯЕВА АНАСТАСИЯ НАГИРОВНА_013431606!1...,00996
997,./data/pdf/СП ТОМСК МАКАРОВА АРИНА АЛЕКСАНДРОВ...,СП ТОМСК МАКАРОВА АРИНА АЛЕКСАНДРОВНА_00316947...,00997


In [3]:
image_info = []

for idx, pdf_path in tqdm(pdf_paths.iterrows()):
    
    if os.path.exists(os.path.join(IMAGES_PATH, pdf_path['pdf_index'] + f'_0000.jpg')):
        continue
        
    images = pyvips.Image.new_from_file(pdf_path['pdf_path'])
    
    for i in range(images.get('n-pages')):
        
        p = os.path.join(IMAGES_PATH, pdf_path['pdf_index'] + f'_{i:04d}.jpg')
        
        image = pyvips.Image.new_from_file(pdf_path['pdf_path'], page=i, dpi=DPI)
        image.write_to_file(p)
        
        info = pdf_path.copy()
        info['image_path'] = os.path.join(IMAGES_PATH, pdf_path['pdf_index'] + f'_{i:04d}.jpg')
        info['image_name'] = pdf_path['pdf_index'] + f'_{i:04d}.jpg'
        image_info.append(info)
        
# image_info = pd.concat(image_info, axis=1, ignore_index=True).T
# image_info.to_parquet('./data/images_info.parquet', index=False, compression='gzip')

0it [00:00, ?it/s]

In [4]:
image_info = pd.read_parquet('./data/images_info.parquet')
image_info.head()

,pdf_path,pdf_name,pdf_index,image_path,image_name
0,./data/pdf/СП ТОМСК ВЕСЕЛОВА ЕКАТЕРИНА МИХАЙЛО...,СП ТОМСК ВЕСЕЛОВА ЕКАТЕРИНА МИХАЙЛОВНА_ПНН8516...,00000,./data/images/00000_0000.jpg,00000_0000.jpg
1,./data/pdf/МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВН...,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,00001,./data/images/00001_0000.jpg,00001_0000.jpg
2,./data/pdf/МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВН...,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,00001,./data/images/00001_0001.jpg,00001_0001.jpg
3,./data/pdf/МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВН...,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,00001,./data/images/00001_0002.jpg,00001_0002.jpg
4,./data/pdf/МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВН...,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,00001,./data/images/00001_0003.jpg,00001_0003.jpg


## Load model

In [5]:
MODEL_PATH = "Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4"

llm = LLM(
    model=MODEL_PATH,
    limit_mm_per_prompt={"image": 1, "video": 1},
    gpu_memory_utilization=0.2,
    max_model_len=6144,
    trust_remote_code=True,
)

sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
    max_tokens=2048,
    stop_token_ids=[],
    seed=42,
)

processor = AutoProcessor.from_pretrained(MODEL_PATH)

WARNING 09-25 18:02:06 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.
INFO 09-25 18:02:08 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-25 18:02:08 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=6144, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marli

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-25 18:02:13 model_runner.py:1008] Loading model weights took 6.4651 GB


/home/nrazuvaev/.conda/envs/env_ocr/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/nrazuvaev/.conda/envs/env_ocr/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 09-25 18:02:16 gpu_executor.py:122] # GPU blocks: 8497, # CPU blocks: 4681
INFO 09-25 18:02:20 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-25 18:02:20 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-25 18:02:45 model_runner.py:1430] Graph capturing finished in 24 secs.


## Config

In [5]:
system_message = '''\
Ты - МТС Финтех OCR Ассистент, который помогает пользователю распознавать текст в документах и обрабатывать результаты.
БУДЬ ОСОБЕННО ВНИМАТЕЛЕН К ИМЕНАМ.
'''

format_fields = lambda fields: ',\n'.join([f'\t"{field}": ...' for field in fields])
answer_format = '''\nОтветь в формате JSON: ```json
{{
{0}
}}
```'''

questions = [
######################################################################################
{'question': '''\
Распознай тип документа.
Тип документа обычно указан сверху страницы крупным шрифтом.
Выбери один из вариантов:
* Судебный приказ
* Определение
* Исполнительный лист
* Письмо (извещение)
* Другое
* Пустой лист\
''', 'fields': ['doc_type'],}, 
######################################################################################
{'question': '''\
Распознай весь текст документа.\
''', 'fields': ['text'],}, 
###################################################################################### 
{'question': '''\
По скану документа и распознанному тексту найди НАИМЕНОВАНИЕ СУДА (СУДЕБНОГО УЧАСТКА).

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['court_name'],}, 
###################################################################################### 
{'question': '''\
По скану документа и распознанному тексту найди НОМЕР ДЕЛА.
Номер дела находится сверху страницы или в правом верхнем углу. Часто начинается с "Дело № ...", "Производство № ...", "№ ...".
В номере дела используются цифры, буквы, дробь (/), часто указан год и т.п. Примеры: "2-1235/16", "2-11-1280/2018", "2г-432-9845/2024-116" и т.п.;

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['case_number'],}, 
######################################################################################
{'question': '''\
По скану документа и распознанному тексту найди ДАТУ ДОКУМЕНТА (ДЕЛА / СУДЕБНОГО ПРОИЗВОДСТВА).
Обычно может находиться сверху страницы слева от номера дела, или по центру;
Преобразуй в формат: %d.%m.%Y

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['case_date'],}, 
######################################################################################
{'question': '''\
По скану документа и распознанному тексту найди НОМЕР КРЕДИТНОГО ДОГОВОРА.

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['credit_id'],}, 
######################################################################################
{'question': '''\
По скану документа и распознанному тексту найди ФАМИЛИЮ, ИМЯ И ОТЧЕСТВО (ФИО) ДОЛЖНИКА.
Преобразуй ФИО в начальную форму, именительный падеж.

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['debtor_full_name'],}, 
######################################################################################
{'question': '''\
По скану документа и распознанному тексту найди КРАТКОЕ РЕШЕНИЕ / ПОСТАНОВЛЕНИЕ ПО ДЕЛУ.
Например, "Взыскать с должника", "Выдать судебный приказ о взыскании", и тому подобное. Более полное решение писать не требуется.

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['result'],}, 
######################################################################################
{'question': '''\
По скану документа и распознанному тексту найди СУММУ ЗАДОЛЖЕННОСТИ ПО КРЕДИТНОМУ ДОГОВОРУ, СУММУ ОСНОВНОГО ДОЛГА, СУММУ ПРОЦЕНТОВ ИЛИ КОМИССИИ, СУММУ ГОСУДАРСТВЕННОЙ ПОШЛИНЫ (ГОС. ПОШЛИНЫ), ОБЩАЯ СУММА (ВСЕГО).
Ответы преобразуй в число с плавающей точкой без пробелов. Например, из "34580 руб. 34 коп." или "тридцать четыре тысячи пятьсот восемьдесят рублей тридцать четыре копейки" в "34580.34".

РАСПОЗНАННЫЙ ТЕКСТ ПОВТОРЯТЬ НЕ ТРЕБУЕТСЯ.
ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['overdue_debt', 'main_debt', 'percent_debt', 'fee', 'total_debt'],}, 
######################################################################################
{'question': '''\
По скану документа найди ДАТУ ВСТУПЛЕНИЯ В СИЛУ ДОКУМЕНТА.
Обычно находится снизу документа в месте печати, с подписью и написана рукописным или печатным текстом.
Рядом со словами, например: "Судебный приказ вступил в силу ..."

ЕСЛИ ИНФОРМАЦИЯ ОТСУТСТВУЕТ ИЛИ НЕ ПОДХОДИТ В ПОЛЕ ЗАПОЛНИТЬ ЗНАЧЕНИЕМ "ПУСТО".\
''', 'fields': ['effective_date'],},
]

prompts = {}
for question in questions:
    question, fields = question['question'], question['fields']
    prompt = f'{question}\n{answer_format.format(format_fields(fields))}'

    prompts[fields[0]] = prompt

In [6]:
prompts.keys()

dict_keys(['doc_type', 'text', 'court_name', 'case_number', 'case_date', 'credit_id', 'debtor_full_name', 'result', 'overdue_debt', 'effective_date'])

In [7]:
images_path  = sorted(glob.glob(os.path.join(IMAGES_PATH, '*.jpg')))

image_name_pattern = '\d+_\d+.jpg'
images_name = [re.findall(image_name_pattern, image_path)[0] for image_path in images_path]

images_path[:3], images_name[:3]

(['./data/images/00000_0000.jpg',
  './data/images/00001_0000.jpg',
  './data/images/00001_0001.jpg'],
 ['00000_0000.jpg', '00001_0000.jpg', '00001_0001.jpg'])

In [8]:
def make_conversation(prompt, image_path, system_message, text=None):
    
    if text is None:
        return [
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image_path,
                        "min_pixels": 224 * 224,
                        "max_pixels": 1280 * 28 * 28,
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ]
    else: 
        return [
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image_path,
                        "min_pixels": 224 * 224,
                        "max_pixels": 1280 * 28 * 28,
                    },
                    {
                        "type": "text", 
                        "text": f'Результат распознавания текста:\n{text}\n\n' + prompt
                    },
                ],
            }
        ]

In [9]:
MAX_NEW_TOKENS = 2048

def run_conversation_hf(conversation):
    text = processor.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(conversation)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    
    return output_text[0]

In [10]:
sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
    max_tokens=2048,
    stop_token_ids=[],
    seed=42,
)

def run_conversation_vllm(conversation):
    text = processor.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(conversation)
    
    mm_data = {}
    if image_inputs is not None:
        mm_data["image"] = image_inputs
    if video_inputs is not None:
        mm_data["video"] = video_inputs

    llm_inputs = {
        "prompt": text,
        "multi_modal_data": mm_data,
    }

    output_text = llm.generate([llm_inputs], sampling_params=sampling_params)
    
    return output_text[0].outputs[0].text

In [11]:
def response_to_json(text):
    text = re.split('```json', text)[1]
    text = re.split('```', text)[0]
    return json.loads(text)

## Run

In [ ]:
# vllm
recognition_results = {}

for image_path, image_name in tqdm(zip(images_path, images_name)):
    
    recognition_results[image_name] = {}
    
    field_name = 'doc_type'
    
    conversation = make_conversation(prompts[field_name], image_path, system_message)
    output_text = run_conversation_vllm(conversation)
    
    try:
        recognition_results[image_name][field_name] = response_to_json(output_text)[field_name]
    except:
        recognition_results[image_name][field_name] = 'Ошибка'
    
    field_name = 'text'
    
    conversation = make_conversation(prompts[field_name], image_path, system_message)
    output_text = run_conversation_vllm(conversation)
    
    try:
        recognition_results[image_name][field_name] = response_to_json(output_text)[field_name]
    except:
        recognition_results[image_name][field_name] = 'Ошибка'
        
    if recognition_results[image_name]['doc_type'] not in ('Судебный приказ', ):
        with open(os.path.join(PROCESSED_LABELS_PATH, image_name.replace('jpg', 'json')), 'w') as f:
            json.dump(recognition_results[image_name], f)
        continue
        
    for field_name in [
        'court_name', 'case_number', 'case_date', 
        'credit_id', 'debtor_full_name', 'result', 
        'overdue_debt', 'effective_date',
    ]:
        if field_name == 'effective_date':
            conversation = make_conversation(prompts[field_name], image_path, system_message)
        else:
            conversation = make_conversation(prompts[field_name], image_path, system_message, recognition_results[image_name]['text'])
        output_text = run_conversation_vllm(conversation)

        try:
            # for field in prompts[field_name]['fields']:
            #     recognition_results[image_name][field_name] = response_to_json(output_text)[field]
            
            res = response_to_json(output_text)
            recognition_results[image_name][field_name] = res[field_name]
            if 'fee' in res:
                recognition_results[image_name]['fee'] = res['fee']
            if 'overdue_debt' in res:
                recognition_results[image_name]['overdue_debt'] = res['overdue_debt']
            if 'main_debt' in res:
                recognition_results[image_name]['main_debt'] = res['main_debt']
            if 'percent_debt' in res:
                recognition_results[image_name]['percent_debt'] = res['percent_debt']
            if 'total_debt' in res:
                recognition_results[image_name]['total_debt'] = res['total_debt']
            
        except:
            recognition_results[image_name][field_name] = 'Ошибка'
    
    with open(os.path.join(PROCESSED_LABELS_PATH, image_name.replace('jpg', 'json')), 'w') as f:
        json.dump(recognition_results[image_name], f)

In [65]:
recognition_results = {}

for image_path, image_name in tqdm(zip(images_path, images_name)):
    
    with open(os.path.join(PROCESSED_LABELS_PATH, image_name.replace('jpg', 'json')), 'r') as f:
        recognition_results[image_name] = json.load(f)
        
df_recognition_results = pd.DataFrame(recognition_results).T

0it [00:00, ?it/s]

In [66]:
df_recognition_results.columns

Index(['doc_type', 'text', 'court_name', 'case_number', 'case_date',
       'credit_id', 'debtor_full_name', 'result', 'overdue_debt', 'fee',
       'main_debt', 'percent_debt', 'total_debt', 'effective_date'],
      dtype='object')

In [67]:
for col in tqdm(df_recognition_results.columns):
    mask = df_recognition_results[col].isin(['Ошибка', '', 'ПУСТО', 'Пусто'])
    df_recognition_results.loc[mask, col] = np.nan

for col in tqdm(['overdue_debt', 'fee', 'main_debt', 'percent_debt', 'total_debt']):
    df_recognition_results[col] = df_recognition_results[col].astype(str).str.replace('None', 'nan', regex=True)
    df_recognition_results[col] = df_recognition_results[col].astype(str).str.replace('р.|руб|коп\.|\s', '', regex=True)
    df_recognition_results[col] = df_recognition_results[col].str.replace(',', '.', regex=True)
    df_recognition_results[col] = df_recognition_results[col].str.replace('б', '6', regex=True)
    
    df_recognition_results[col] = pd.to_numeric(df_recognition_results[col], errors='coerce')
    df_recognition_results.loc[df_recognition_results[col].eq(0), col] = np.nan

for col in tqdm(['case_date']):
    mask_8 = df_recognition_results[col].astype(str).str.len().eq(8)
    df_recognition_results.loc[mask_8, col] =\
    pd.to_datetime(df_recognition_results.loc[mask_8, col], format='%d.%m.%y', errors='coerce')
    
    mask_10 = df_recognition_results[col].astype(str).str.len().eq(10)
    df_recognition_results.loc[mask_10, col] =\
    pd.to_datetime(df_recognition_results.loc[mask_10, col], format='%d.%m.%Y', errors='coerce')
    
    df_recognition_results.loc[~(mask_8 | mask_10), col] = pd.NaT
    df_recognition_results[col] = df_recognition_results[col].replace(np.nan, pd.NaT)
    
for col in tqdm(['credit_id', 'case_number']):
    pattern_date = '\d+\.\d+\.\d+'
    pattern_number_sign = '№'
    pattern_passport = '\d{4} \d{6}|\d{2} \d{2} \d{6}'
    pattern_str = 'Дело'
    
    df_recognition_results[col] = df_recognition_results[col].astype(str)\
    .str.replace(f'{pattern_date}|{pattern_number_sign}|{pattern_passport}|{pattern_str}', '', regex=True)
    
    df_recognition_results[col] = df_recognition_results[col]\
    .str.replace(f'\s+', '', regex=True)
    
    mask = df_recognition_results[col].str.contains('год|nan') | df_recognition_results[col].eq('')
    df_recognition_results.loc[mask, col] = None
    
    df_recognition_results[col] = df_recognition_results[col]
    
for col in tqdm(['doc_type']):
    mask = df_recognition_results['doc_type'].eq('Судебный приказ') &\
            df_recognition_results['overdue_debt'].isna() &\
            (df_recognition_results['debtor_full_name'].astype(str).str.contains('\w\.\s*\w\.', regex=True) |\
             df_recognition_results['debtor_full_name'].isna() |\
             df_recognition_results['debtor_full_name'].astype(str).fillna('').str.split(' ').apply(len).lt(2) |\
             df_recognition_results['case_number'].astype(str).apply(len).le(8) |\
             df_recognition_results['credit_id'].astype(str).apply(len).le(10)
            )
    
    df_recognition_results.loc[mask, col] = 'Другое'
    
for col in tqdm(df_recognition_results.columns):
    display(df_recognition_results[col].value_counts(dropna=False).to_frame())


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_3503091/789273918.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_recognition_results[col] = df_recognition_results[col].replace(np.nan, pd.NaT)


  0%|          | 0/2 [00:00<?, ?it/s]

...

,count
fee,
NaN,1265
200.00,30
582.31,3
10384.00,3
433.50,3
...,...
763.99,1
1101.52,1
437.50,1


,count
main_debt,
NaN,1183
34580.34,14
20000.00,6
32154.00,2
7329.68,2
...,...
29002.99,1
78474.93,1
18021.46,1


,count
percent_debt,
NaN,1926
12154.00,9
10384.00,5
13983.00,3
11706.00,2
...,...
12240.84,1
773.11,1
2240.44,1


,count
total_debt,
NaN,1181
34580.34,16
123456.78,2
22875.14,2
32736.31,2
...,...
33517.55,1
25157.57,1
59240.94,1


,count
effective_date,
NaN,967
2023,40
29 сентября 2023 года,22
2024,19
26 сентября 2023 года,17
...,...
24 10 2023,1
10 2023 г.,1
14 Авт 2024,1


In [68]:
output = image_info.set_index('image_name').join(df_recognition_results)
output.drop(columns = ['pdf_path', 'pdf_index', 'image_path'], inplace=True)
# output.to_excel('ocr-result-2024-09-26.xlsx')
output

,pdf_name,doc_type,text,court_name,case_number,case_date,credit_id,debtor_full_name,result,overdue_debt,fee,main_debt,percent_debt,total_debt,effective_date
image_name,,,,,,,,,,,,,,,
00000_0000.jpg,СП ТОМСК ВЕСЕЛОВА ЕКАТЕРИНА МИХАЙЛОВНА_ПНН8516...,Судебный приказ,Мировой судья судебного участка № 7 Нижневарто...,Мировой судья судебного участка № 7 Нижневарто...,02-5397/2107/2023,2023-10-05,ПНН851674/810/21,Веселова Екатерина Михайловна,Взыскать с должника,19227.55,384.55,19227.55,NaN,19612.10,5 октября 2023 года
00001_0000.jpg,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,Письмо (извещение),Письмо заказное\nС электронным уведомлением\nС...,NaN,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00001_0001.jpg,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,Пустой лист,0123456789,NaN,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00001_0002.jpg,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,Судебный приказ,Российская Федерация\nМировой судья судебного ...,Мировой судья судебного участка № 5 Фрунзенско...,2-1114/2024,2024-05-27,000562458/105/23,Слабжина Анастасия Алексеевна,Взыскать с Слабжиной Анастасии Алексеевны в по...,20037.00,400.56,20037.00,NaN,20437.56,25.06.2024
00001_0003.jpg,МСК СП САВИНОВА АНАСТАСИЯ АЛЕКСЕЕВНА_000562458...,Пустой лист,NaN,NaN,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00998_0001.jpg,МСК СП ДАВТЯН ТАТЬЯНА СЕРГЕЕВНА_022541405!105!...,Пустой лист,NaN,NaN,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00998_0002.jpg,МСК СП ДАВТЯН ТАТЬЯНА СЕРГЕЕВНА_022541405!105!...,Судебный приказ,Судебный участок № 52 Алданского района Респуб...,Судебный участок № 52 Алданского района Респуб...,2-643/52-2024,2024-03-22,022541405/105/23,Давтян Татьяна Сергеевна,Взыскать с должника Давтян Татьяны Сергеевны,24515.47,467.73,24515.47,NaN,24983.20,15.04.2024
00998_0003.jpg,МСК СП ДАВТЯН ТАТЬЯНА СЕРГЕЕВНА_022541405!105!...,Пустой лист,NaN,NaN,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
def verify_string(row, s1='client_full_name', s2='debtor_full_name', threshold=2):
    if isinstance(row[s1], str) and isinstance(row[s2], str):
        return nltk.edit_distance(row[s1], row[s2].upper()) <= threshold
    return False

def extract_client_name(text):
    text = re.split('СП ТОМСК |МСК СП |РЕГ СП |ОНЛАЙН СП ', text)[1]
    text = re.split('_', text)[0]
    return text

def extract_credit_id(text):
    text = re.split('_', text)[1]
    text = text.replace('!', '/')
    return text

output['client_name'] = output['pdf_name'].apply(extract_client_name)
output['credit_id_true']   = output['pdf_name'].apply(extract_credit_id)

In [72]:
mask = output['doc_type'].eq('Судебный приказ')

value = output[mask].apply(verify_string, axis=1, s1='client_name', s2='debtor_full_name').sum()
print(f'Кол-во СП: {mask.sum()}, кол-во совпавших: {value}')
value /= mask.sum()
print(f'Доля совпавших ФИО по СП: {round(100 * value, 2)}%',)

Кол-во СП: 1090, кол-во совпавших: 983
Доля совпавших ФИО по СП: 90.18%


In [80]:
mask = output['doc_type'].eq('Судебный приказ')

value = output[mask].apply(verify_string, axis=1, s1='credit_id_true', s2='credit_id', threshold=0).sum()
print(f'Кол-во СП: {mask.sum()}, кол-во совпавших: {value}, кол-во пустых: {output.loc[mask, "credit_id"].isna().sum()}')
value /= mask.sum()
print(f'Доля совпавших номеров КД по СП: {round(100 * value, 2)}%',)

Кол-во СП: 1090, кол-во совпавших: 803, кол-во пустых: 32
Доля совпавших номеров КД по СП: 73.67%
